# Optical Flow

This notebook demonstrates the use of the optical flow method to extract the breathing signal from a video. The optical flow method is based on the movement of feature points in the video frames. The movement of the feature points is tracked over time, and the amplitude of the movement is used to extract the breathing signal.

In [ ]:
import respiration.dataset as repository

dataset = repository.from_default()

subject = 'Proband16'
setting = '101_natural_lighting'

subject_frames, params = dataset.get_video_gray(subject, setting)

In [ ]:
hyperparameters = {
    'quality_level': 0.1,
    'quality_level_rv': 0.05,
    'filter_lowpass': 0.1,
    'filter_highpass': 0.6,
}

In [ ]:
import numpy as np
import respiration.roi as roi

yolo = roi.YOLO()


def find_roi(frame: np.ndarray) -> tuple[int, int, int, int]:
    """
    Find the region of interest (ROI) based on the face detection
    :param frame: The frame to find the ROI
    :return: The region of interest (ROI) coordinates (x, y, w, h)
    """

    persons = yolo.detect_classes(frame, 'person')
    return persons[0] if len(persons) > 0 else None

    # faces = utils.detect_faces(frame)
    # 
    # if len(faces) == 0:
    #     raise ValueError('No face detected in the first frame')
    # elif len(faces) > 1:
    #     raise ValueError('Multiple faces detected in the first frame')
    # 
    # return utils.roi_from_face(faces[0], scale_w=0.2, scale_h=0.2)

In [ ]:
frame1 = subject_frames[0]
subject_roi = find_roi(frame1)

In [ ]:
import matplotlib.pyplot as plt
from respiration.extractor import optical_flow

# Get different feature points for the first frame
default_points = optical_flow.select_feature_points(frame1)
special_points = optical_flow.select_feature_points(frame1, fpn=5)
roi_points = optical_flow.select_feature_points(frame1, roi=subject_roi)
special_roi = optical_flow.select_feature_points(frame1, roi=subject_roi, fpn=5)

# Plot the first frame with the feature points
plt.imshow(frame1, cmap='gray')

# Draw the region of interest (ROI)
roi_x, roi_y, roi_w, roi_h = subject_roi
plt.gca().add_patch(plt.Rectangle(
    (roi_x, roi_y), roi_w, roi_h,
    linewidth=1, edgecolor='r', facecolor='none'))

for iny in range(default_points.shape[0]):
    plt.scatter(default_points[iny, 0, 0],
                default_points[iny, 0, 1],
                c='r', s=2.5)

for iny in range(special_points.shape[0]):
    plt.scatter(special_points[iny, 0, 0],
                special_points[iny, 0, 1],
                c='b', s=2.5)

for iny in range(roi_points.shape[0]):
    plt.scatter(roi_points[iny, 0, 0],
                roi_points[iny, 0, 1],
                c='#FFFF00', s=2.5)

for iny in range(special_roi.shape[0]):
    plt.scatter(special_roi[iny, 0, 0],
                special_roi[iny, 0, 1],
                c='#FF00FF', s=2.5)

plt.show()

In [ ]:
# Track the movement of the feature points
feature_point_movements = optical_flow.track_feature_point_movement(subject_frames, special_points)

# Extract the amplitudes of the feature points
raw_signal = optical_flow.calculate_feature_point_amplitudes(feature_point_movements)

In [ ]:
# Plot the raw signal
plt.figure(figsize=(20, 6))
plt.plot(raw_signal)

In [ ]:
import respiration.preprocessing as preprocessing

preprocessed_unprocessed = optical_flow.signal_from_amplitudes(
    raw_signal,
    use_cgof=False,
)

signal_cgof = optical_flow.signal_from_amplitudes(
    raw_signal,
    use_cgof=True,
)
signal_filter = preprocessing.butterworth_filter(
    signal_cgof,
    params.fps,
    lowpass=hyperparameters['filter_lowpass'],
    highpass=hyperparameters['filter_highpass'],
)
signal_normalization = preprocessing.normalize_signal(
    signal_filter,
)

processed_signal = signal_normalization

In [ ]:
import matplotlib.pyplot as plt

fig, axs = plt.subplots(3, 1, figsize=(15, 10))

axs[0].plot(preprocessed_unprocessed)
axs[0].set_title('Unprocessed')

axs[1].plot(signal_cgof)
axs[1].set_title('CGOF')

axs[2].plot(signal_filter)
axs[2].set_title('Filter')

plt.show()

In [ ]:
from scipy import signal

gt_signal, gt_sample_rate = dataset.get_ground_truth_rr_signal(subject, setting)

gt_signal = preprocessing.butterworth_filter(
    gt_signal,
    gt_sample_rate,
    lowpass=hyperparameters['filter_lowpass'],
    highpass=hyperparameters['filter_highpass'],
)
gt_signal = preprocessing.normalize_signal(gt_signal)

In [ ]:
# Plot the processed_signal and the gt_signal in the same plot without subplots
plt.figure(figsize=(15, 5))
plt.title('Processed Signal vs Ground Truth Signal')

plt.plot(processed_signal, label='Processed Signal')

# Down sample the ground truth signal to the same FPS
gt_signal_down = signal.resample(gt_signal, len(processed_signal))
plt.plot(gt_signal_down, label='Ground Truth Signal')

plt.legend()
plt.show()

In [ ]:
from respiration.analysis import frequency_extractor

# Calculate the frequencies using the different methods for the ground truth signal
frequency = frequency_extractor.FrequencyExtractor(
    gt_signal,
    gt_sample_rate,
    lowpass=hyperparameters['filter_lowpass'],
    highpass=hyperparameters['filter_highpass'],
)

gt_frequency = {
    'FFT': frequency.frequency_from_fft(),
    'PC': frequency.frequency_from_peaks(),
    'CP': frequency.frequency_from_crossing_point(),
    'NFCP': frequency.frequency_from_nfcp()
}
gt_frequency

In [ ]:
import pandas as pd

evaluation_results = []

for fpn in [None, 5]:
    for use_roi in [False, True]:
        if use_roi:
            roi = subject_roi
        else:
            roi = None

        breathing_signal = optical_flow.extract_signal(
            subject_frames,
            fpn=fpn,
            quality_level=hyperparameters['quality_level'],
            quality_level_rv=hyperparameters['quality_level_rv'],
            roi=roi,
        )

        frequency = frequency_extractor.FrequencyExtractor(
            breathing_signal,
            params.fps,
            lowpass=hyperparameters['filter_lowpass'],
            highpass=hyperparameters['filter_highpass'],
        )

        predictions = {
            'FFT': frequency.frequency_from_fft(),
            'PC': frequency.frequency_from_peaks(),
            'CP': frequency.frequency_from_crossing_point(),
            'NFCP': frequency.frequency_from_nfcp(),
        }

        for method, value in predictions.items():
            evaluation_results.append({
                'fpn': fpn if fpn is not None else 'None',
                'use_roi': use_roi,
                'method': method,
                'value': value,
                'error': abs(value - gt_frequency[method]),
            })

In [ ]:
evaluation = pd.DataFrame(evaluation_results)
evaluation

In [ ]:
fig, axs = plt.subplots(4, 1, figsize=(15, 10))

# Add more space between the subplots
fig.subplots_adjust(hspace=0.5)

inx = 0
for method in gt_frequency.keys():
    axs[inx].set_title(f'{method} Method')

    # Plot the values for each method
    data = evaluation[evaluation['method'] == method]

    title = 'fpn=' + data['fpn'].astype(str) + ' / roi=' + data['use_roi'].astype(str)

    # Error in Beats Per Minutes (BPM)
    axs[inx].bar(title, data['error'] * 60)

    inx += 1